In [140]:
#!pip install nltk
#!pip install gensim
#!pip install scikit-learn
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [141]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
import gensim
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
import gensim.downloader as api
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import random
import itertools
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
import torch
import matplotlib.pyplot as plt
import matplotlib
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import logging

In [142]:
print("gensim = " + gensim.__version__)
print("numpy = " + np.__version__)
print("pandas = " + pd.__version__)
print("pytorch = " + torch.__version__)

gensim = 4.1.2
numpy = 1.21.5
pandas = 1.4.4
pytorch = 2.2.2+cu118


In [143]:
df_IoT=pd.read_pickle("./data/IoT_data/IotDataClean.pkl")
df_kdd=pd.read_pickle("./data/IoT_data/kddDataClean.pkl")

In [144]:
df_IoT.head()

,MI_dir_L5_weight,MI_dir_L5_mean,MI_dir_L5_variance,MI_dir_L3_weight,MI_dir_L3_mean,MI_dir_L3_variance,MI_dir_L1_weight,MI_dir_L1_mean,MI_dir_L1_variance,MI_dir_L0.1_weight,...,HpHp_L0.1_covariance,HpHp_L0.1_pcc,HpHp_L0.01_weight,HpHp_L0.01_mean,HpHp_L0.01_std,HpHp_L0.01_magnitude,HpHp_L0.01_radius,HpHp_L0.01_covariance,HpHp_L0.01_pcc,Attack Type
49533,0.343251,2.437210,0.063571,0.507774,2.235071,0.620085,0.756944,1.827138,1.220649,0.996901,...,-0.011188,-0.016171,-0.184341,2.036068,-0.025337,2.011124,-0.012603,-0.01115,-0.032123,0
93899,-1.148067,-0.439153,-0.613534,-1.149546,-0.459583,-0.639390,-1.142232,-0.477132,-0.653180,-1.135319,...,-0.011188,-0.016171,-0.183005,-0.350359,-0.025336,-0.189836,-0.012603,-0.01115,-0.032123,1
56100,-1.119081,-0.411869,-0.613534,-1.131234,-0.431286,-0.639390,-1.135898,-0.448229,-0.653180,-1.134690,...,-0.011188,-0.016171,-0.183341,-0.329426,-0.025337,-0.159864,-0.012603,-0.01115,-0.032123,1
188917,-1.148067,-0.698352,-0.613534,-1.149546,-0.728404,-0.639390,-1.142232,-0.751711,-0.653180,-1.135659,...,-0.011188,-0.016171,-0.184341,-0.549228,-0.025337,-0.606246,-0.012603,-0.01115,-0.032123,1
60741,1.264267,0.539349,1.821256,1.188261,0.916127,1.794928,1.026018,1.364279,1.602694,1.008771,...,-0.011188,-0.016171,-0.184341,-0.549228,-0.025337,-0.606246,-0.012603,-0.01115,-0.032123,1


In [145]:
df_kdd.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,pt_vec,pt_svc,pt_flg,pt_tgt,Attack Type
454816,-0.067792,-0.002017,-0.026287,-0.006673,-0.04772,-0.002571,-0.044136,-0.009782,-0.417192,-0.005679,...,-0.158629,-0.464418,-0.463202,-0.25204,-0.249464,-0.821297,0.742168,0.0,0.066928,1
96364,-0.067792,-0.002559,-0.017843,-0.006673,-0.04772,-0.002571,-0.044136,-0.009782,2.396980,-0.005679,...,-0.158629,-0.464418,-0.463202,-0.25204,-0.249464,0.955787,1.435111,0.0,0.066928,1
454549,-0.067792,-0.002017,-0.026287,-0.006673,-0.04772,-0.002571,-0.044136,-0.009782,-0.417192,-0.005679,...,-0.158629,-0.464418,-0.463202,-0.25204,-0.249464,-0.821297,0.742168,0.0,0.066928,1
195889,-0.067792,-0.002017,-0.026287,-0.006673,-0.04772,-0.002571,-0.044136,-0.009782,-0.417192,-0.005679,...,-0.158629,-0.464418,-0.463202,-0.25204,-0.249464,-0.821297,0.742168,0.0,0.066928,1
437071,-0.067792,-0.002777,0.031310,-0.006673,-0.04772,-0.002571,-0.044136,-0.009782,2.396980,-0.005679,...,-0.158629,-0.464418,-0.463202,-0.25204,-0.249464,0.955787,-1.086324,0.0,0.066928,1


In [146]:
X = df_kdd.iloc[:, :-1]  # All rows, all columns except the last one
y = df_kdd.iloc[:, -1]   # All rows, only the last column
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train_list = X_train.values.tolist()
y_train_list = y_train.values.tolist()
X_test_list = X_test.values.tolist()
y_test_list = y_test.values.tolist()
print(type(y_train_list[0]))

<class 'int'>


In [147]:
class DataWrapper(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = torch.tensor(self.data[idx], dtype=torch.float32)
        y = torch.tensor(self.labels[idx], dtype=torch.float32)
        return x,y 
    
    def avgLen(self):
      lens = [len(x) for x in self.data]
      return sum(lens)/len(lens)

In [148]:
train_data = DataWrapper(X_train_list, y_train_list)
test_data = DataWrapper(X_test_list, y_test_list)

In [149]:
# X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
# y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
# X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
# y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
# test_dataset = torch.utils.data.TensorDataset(X_test_tensor, y_test_tensor)

In [150]:
train_loader = DataLoader(dataset=train_data, batch_size=32, shuffle=False)
test_loader = DataLoader(dataset=test_data, batch_size=100, shuffle=False)

print(train_loader)

In [151]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        # Define the LSTM layer
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        
        # Define the output layer
        self.linear = nn.Linear(hidden_dim, output_dim)
    def forward(self, x):

        _, (hidden, _) = self.lstm(x)

        # if isinstance(hidden, tuple):
        #    hidden = hidden[0]
        # hidden = hidden[-1, :, :]

        # Forward pass through the linear layer
        out = self.linear(hidden[-1])

        return out

In [152]:
def test_metrics(model, loader):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_ae = 0.0
    criterion = torch.nn.MSELoss()
    for i, (x, y) in enumerate(loader):
        x = x.cuda()
        y = y.cuda()

        y_hat = model(x)

        # Reshape the target tensor to match the shape of the input tensor
        y = y.view(-1, 1)

        loss = criterion(y_hat, y)

        pred = (y_hat > 0.5).float()

        correct += (pred == y).float().sum()
        total += y.shape[0]
        sum_loss += loss * y.shape[0]
    return (sum_loss / total).item(), (correct / total).item()

In [153]:
input_dim = X_train.shape[1]  # number of input features
hidden_dim = 50
num_layers = 2
output_dim = 1

model = LSTMModel(input_dim, hidden_dim, num_layers, output_dim)
model.cuda()
criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [154]:
num_epochs = 50

plot_data = {"train": [], "test": []}


for epoch in range(num_epochs):
    model.train()
    sum_loss = 0.0
    total = 0
    correct = 0

    plot_data = {"train": [], "test": []}
    iter = 0

    for j, (x, y) in enumerate(train_loader):
        x=x.cuda()
        y=y.cuda()
        optimizer.zero_grad()

        # Forward pass
        outputs = model(x)
        # Reshape the target tensor to match the shape of the input tensor
        y = y.view(-1, 1)

        loss = criterion(outputs, y)
        
        # Backward and optimize
        loss.backward()
        optimizer.step()

        pred = (outputs > 0.5).float()
        correct += (pred == y).float().sum()
        sum_loss += loss.item() * y.shape[0]
        total += y.shape[0]

        iter += 1
        plot_data['train'].append((iter, loss.item(), (pred == y).float().mean().item()))

    test_loss, test_acc = test_metrics(model, test_loader)

    logging.warning("[Epoch {:3}]   Loss:  {:8.4}     Train Acc:  {:8.4}%     Test Acc:  {:8.4}%".format(epoch + 1,
                                                                                                             sum_loss / total,
                                                                                                             correct / total * 100,
                                                                                                             test_acc * 100))

    plot_data['test'].append((iter, test_loss, test_acc))

c:\Users\alexa\anaconda3\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


KeyboardInterrupt: 